<a href="https://colab.research.google.com/github/nyanta012/demo/blob/main/faster_whisper_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Google Colabの使い方

- **セルの実行:** `Ctrl + Enter`, `Shift + Enter`
- **セルの作成:** `Ctrl + M B`
- **セルの削除:** `Ctrl + M D`

In [ ]:
%%capture
!pip install faster-whisper==0.10.0

# Google Drive連携

In [ ]:
# prompt: google driveにアクセスして動画ファイルを取得
import os
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
os.listdir("/content/drive/MyDrive/video/")

['test.mp4']

In [ ]:
path_video = "/content/drive/MyDrive/video/test.mp4"

# Faster Whisper-v3ダウンロード


In [ ]:
# srt file を作成するための関数
def create_srt_file(results, use_faster_whisper, file_name="transcribe"):
    data = []
    with open(f"{file_name}.srt", mode="w") as f:
        for index, _dict in enumerate(results):
            if use_faster_whisper:
              start_time = _dict.start # start
              end_time = _dict.end # end
              text = _dict.text # text
            else:
              start_time = _dict["start"]
              end_time = _dict["end"]
              text = _dict["text"]

            data.append({
            "index": index + 1,
            "start": start_time,
            "end": end_time,
            "text": text})

            # 時、分、秒、ミリ秒に分割
            s_h, s_m, s_s = int(start_time // 3600), int((start_time % 3600) // 60), int(start_time % 60)
            e_h, e_m, e_s = int(end_time // 3600), int((end_time % 3600) // 60), int(end_time % 60)

            # ミリ秒を計算
            s_ms = int((start_time - int(start_time)) * 1000)
            e_ms = int((end_time - int(end_time)) * 1000)

            f.write(f"{index+1}\n")
            f.write(f"{s_h:02}:{s_m:02}:{s_s:02},{s_ms:03} --> {e_h:02}:{e_m:02}:{e_s:02},{e_ms:03}\n")
            f.write(f"{text}\n\n")
    return data

In [ ]:
import torch
import gc

def release_model_memory(model):
    """
    指定されたモデルをメモリから削除し、ガーベージコレクションとPyTorchのキャッシュメモリ解放を行う関数。

    Parameters:
    model (torch.nn.Module): メモリから解放するモデルオブジェクト。
    """
    # モデルを削除
    del model

    # ガーベージコレクションを実行してメモリを解放
    gc.collect()

    # PyTorchのキャッシュされたメモリを解放
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

In [ ]:
from faster_whisper import WhisperModel

In [ ]:
# model = WhisperModel("large-v3", device="cuda", compute_type="int8_float16")
model = WhisperModel("large-v3", device="cuda", compute_type="float16")

vocabulary.json:   0%|          | 0.00/1.07M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/2.39k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

model.bin:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

In [ ]:
%%time
segments, info = model.transcribe(path_video, beam_size=5)

CPU times: user 2.98 s, sys: 841 ms, total: 3.82 s
Wall time: 12.6 s


In [ ]:
%%time
faster_whisper_data = create_srt_file(results=segments, use_faster_whisper=True, file_name="transcribe_faster_whisper")

CPU times: user 36.1 s, sys: 218 ms, total: 36.3 s
Wall time: 30.1 s


# VAD filter

In [ ]:
%%time
segments, info = model.transcribe(path_video, beam_size=5, vad_filter=True, vad_parameters=dict(min_silence_duration_ms=300))

CPU times: user 2.98 s, sys: 117 ms, total: 3.09 s
Wall time: 3.27 s


In [ ]:
%%time
faster_whisper_vad_filter_data_300 = create_srt_file(results=segments, use_faster_whisper=True, file_name="transcribe_faster_whisper_vad_filter")

CPU times: user 35.8 s, sys: 366 ms, total: 36.1 s
Wall time: 31.7 s


# whisper-v3

※ここからは検証用なので実行しなくても良いです

In [ ]:
# メモリ解放
release_model_memory(model)

In [ ]:
%%capture
!pip install openai-whisper==20231106

In [ ]:
# prompt: whisperをpythonで書く
import whisper

model = whisper.load_model("large-v3")

100%|█████████████████████████████████████| 2.88G/2.88G [01:07<00:00, 45.5MiB/s]


In [ ]:
%%time
result = model.transcribe(audio=path_video, fp16=True)

CPU times: user 1min 8s, sys: 365 ms, total: 1min 9s
Wall time: 1min 17s


In [ ]:
%%time
normal_whisper = create_srt_file(results=result["segments"], use_faster_whisper=False, file_name="transcribe_whisper")

CPU times: user 1.15 ms, sys: 17 µs, total: 1.17 ms
Wall time: 2.35 ms


# 結果の比較

In [ ]:
import pandas as pd

df_whisper = pd.DataFrame(normal_whisper)
df_faster_whisper = pd.DataFrame(faster_whisper_data)

In [ ]:
df_merge = pd.merge(df_faster_whisper[["index", "start", "end", "text"]],
                     df_whisper[["index", "start", "end", "text"]],
                     how="outer", on="index", suffixes=('_faster', '_normal'))

In [ ]:
df_merge.head(50)

,index,start_faster,end_faster,text_faster,start_normal,end_normal,text_normal
0,1,0.00,3.86,このアシスタントAPIを使うには最初にまずアシスタントというのを作ります,0.00,3.86,このアシスタントAPIを使うには最初にまずアシスタントというのを作ります
1,2,3.86,6.80,オリジナルのGPTが作れますよという感じですね,3.86,6.80,オリジナルのGPTが作れますよという感じですね
2,3,6.80,8.86,皆さんこんにちは、AI部長にゃんたです,6.80,8.86,皆さんこんにちは、AI部長にゃんたです
3,4,8.86,11.64,本日はオープンAIデブデイということで,8.86,11.64,本日はオープンAIデブデイということで
4,5,11.64,14.44,オープンAIから新しい機能がたくさん出てきたので,11.64,14.44,オープンAIから新しい機能がたくさん出てきたので
5,6,14.44,15.98,そちらについてまとめていきます,14.44,15.98,そちらについてまとめていきます
6,7,15.98,19.40,日本時間の11月7日の午前3時から,15.98,19.40,日本時間の11月7日の午前3時から
7,8,19.40,22.32,デベロッパーズコンファレンスというのが開催されてたんですが,19.40,22.32,Developers Conferenceというのが開催されていたんですが
8,9,22.32,23.60,皆さん見てみましたか?,22.32,23.60,皆さん見てみましたか?
9,10,24.00,26.20,午前3時からだったのでにゃんたはですね,24.00,26.20,午前3時からだったのでにゃんたはですね


# VAD Filterを使った場合

In [ ]:
df_faster_whisper_vadfilter_300 = pd.DataFrame(faster_whisper_vad_filter_data_300)

In [ ]:
df_merge = pd.merge(df_faster_whisper[["index", "start", "end", "text"]],
                     df_faster_whisper_vadfilter_300[["index", "start", "end", "text"]],
                     how="outer", on="index", suffixes=('_faster', '_vadfilter_300'))

In [ ]:
df_merge.head(50)

,index,start_faster,end_faster,text_faster,start_vadfilter_300,end_vadfilter_300,text_vadfilter_300
0,1,0.00,3.86,このアシスタントAPIを使うには最初にまずアシスタントというのを作ります,0.00,3.86,このアシスタントAPIを使うには最初にまずアシスタントというのを作ります
1,2,3.86,6.80,オリジナルのGPTが作れますよという感じですね,3.86,6.80,オリジナルのGPTが作れますよという感じですね
2,3,6.80,8.86,皆さんこんにちは、AI部長にゃんたです,6.80,8.86,皆さんこんにちは、AI部長にゃんたです
3,4,8.86,11.64,本日はオープンAIデブデイということで,8.86,11.64,本日はオープンAIデブデイということで
4,5,11.64,14.44,オープンAIから新しい機能がたくさん出てきたので,11.64,14.44,オープンAIから新しい機能がたくさん出てきたので
5,6,14.44,15.98,そちらについてまとめていきます,14.44,15.98,そちらについてまとめていきます
6,7,15.98,19.40,日本時間の11月7日の午前3時から,15.98,19.40,日本時間の11月7日の午前3時から
7,8,19.40,22.32,デベロッパーズコンファレンスというのが開催されてたんですが,19.40,22.32,デベロッパーズコンファレンスというのが開催されてたんですが
8,9,22.32,23.60,皆さん見てみましたか?,22.32,23.60,皆さん見てみましたか?
9,10,24.00,26.20,午前3時からだったのでにゃんたはですね,24.00,26.20,午前3時からだったのでにゃんたはですね
